### Getting started with langchain

In [1]:
%pip install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/d5/d0/5e0bbcabe42f9e09c66fa0964b2ecdfee9a92cb2d7f05dd340b93203a40a/langchain-0.1.14-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/91/ca/7219b838086086972e662c19e908694bdc6744537fb41b70392501b8b5e4/dataclasses_json-0.6.4-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langchain-community<0.1,>=0.0.30 from https://files.pythonhosted.org/packages/8a/67/6855b4a55e4e150264fd1a51888d20a1ba3c1b7757660f7ec4a14bd3d7a0/langchain_community-0.0.31-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.2.0,>=0.1.37 from https://files.pythonhosted.org/packages

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [4]:
llm.invoke("how can langsmith help with testing?")

'\nLangSmith is a language model that can assist with testing in several ways:\n\n1. Automated Testing: LangSmith can be used to generate test cases automatically, based on the API documentation or specifications of the system being tested. This can save time and effort compared to writing manual test cases.\n2. Smart Bounding: LangSmith can be used to generate test cases that cover a wide range of inputs and scenarios, including edge cases and corner cases. This can help ensure that the system is tested thoroughly and that no critical bugs are missed.\n3. Test Data Generation: LangSmith can be used to generate test data automatically, such as input values or test cases. This can be especially useful when dealing with large datasets or complex systems where it is difficult to come up with a complete set of test cases manually.\n4. Code Coverage Analysis: LangSmith can be used to analyze the code coverage of the system being tested, identifying areas that are not being tested adequately

### Use langchain summary 'stuff'

In [5]:
%pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain

# Set env var OPENAI_API_KEY or load from a .env file
# import dotenv

# dotenv.load_dotenv()


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

# llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
llm = Ollama(model="llama2")
chain = load_summarize_chain(llm, chain_type="stuff")

summary = chain.invoke(docs)

In [14]:
len(docs[0].page_content)

43594

In [23]:
summary.keys()

dict_keys(['input_documents', 'output_text'])

In [27]:
len(summary['input_documents'][0].page_content)

43594

In [21]:
len(summary['output_text'])

1042

### Use Map-Reduce in langchain summary

In [35]:
%pip install transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/15/fc/7b6dd7e1adc0a6407b845ed4be1999e98b6917d0694e57316d140cc85484/transformers-4.39.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 962.7 kB/s eta 0:00:00m eta 0:00:010:01:01
  Obtaining dependency information for safetensors>=0.4.1 from https://files.pythonhosted.org/packages/03/a0/67f2ed19dbc609ce1bf561a285a6cc8f804846a7f4d9c46b403be93022e4/safetensors-0.4.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 12.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.4/393.4 kB 13.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

In [30]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter

llm = Ollama(model="llama2")
# llm = ChatOpenAI(temperature=0)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [31]:
# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [32]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [33]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [36]:
print(map_reduce_chain.run(split_docs))

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (3902 > 1024). Running this sequence through the model will result in indexing errors


The main themes that emerge from the provided list of documents can be consolidated as follows:

1. Language Models (LMs): The use of LMs for various tasks such as reasoning, problem-solving, and generating text is a recurring theme throughout the documents.
2. Autonomous Agents: The development of autonomous agents that can perform tasks independently, such as prompting, deliberation, and decision-making, is another major theme.
3. Steerability: The ability to guide or control a language model's behavior through explicit inputs or cues is discussed in several documents.
4. Reasoning and Problem-Solving: The use of LMs for reasoning and problem-solving tasks, such as solving complex problems, chain of thought prompting, and tree of thoughts, is also a common theme.
5. Planning and Proficiency: The goal of improving the performance of LMs through various techniques, such as using large language models (LLMs) or incorporating external knowledge sources, is discussed in some documents.
6.

In [46]:
import requests
import json

prompt = """
What is AI?
Can you explain in three paragraphs?
"""

# url = 'http://35.239.223.90:8501/api/chat'
# url = 'http://172.28.0.12:8501/api/chat'
url = 'https://merger-speech-momentum-allocation.trycloudflare.com/api/chat'
payload = {
    "model": "mistral",
    "temperature": 0.6,
    "stream": False,
    "messages": [
        {"role": "system", "content": "You are an AI assistant!"},
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(url, json=payload)
message_str = response.content.decode('utf-8')
message_dict = json.loads(message_str)
print(message_dict['message']['content'])

JSONDecodeError: Extra data: line 1 column 5 (char 4)

In [47]:
message_str

'502 Bad Gateway\nUnable to reach the origin service. The service may be down or it may not be responding to traffic from cloudflared\n'

In [42]:
import json
message_dict = json.loads(message_str)
print(message_dict['message']['content'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)